In [ ]:
import math

from mongoengine import connect, Document, EmbeddedDocument, EmbeddedDocumentField, StringField, \
  IntField, ListField, ValidationError

class Tarjeta(EmbeddedDocument):
    nombre=StringField()
    numero=IntField()
    mes=IntField()
    year=IntField()
    cvv=IntField()

class Producto(Document):
    codigo_barras=StringField()
    nombre=StringField()
    categoria_principal=IntField()
    categorias_secundarias=ListField()

class Linea(EmbeddedDocument):
    num_items=IntField()
    precio_item=IntField()
    nombre_item=StringField()
    total=IntField()
    producto=EmbeddedDocumentField(Producto, required=False)

class Pedido(Document):
    total=IntField()
    fecha=StringField()
    lineas=ListField()

class Usuario(Document):
    dni=StringField()
    nombre=StringField()
    apellido1=StringField()
    apellido2=StringField()
    f_nac=StringField()
    tarjetas=ListField()
    pedidos=ListField()
